написать класс для запросов

Do not contact the server more often than once every 10 seconds.\
Do not poll for any single RID more often than once a minute.\
Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.\
Run scripts weekends or between 9 pm and 5 am Eastern time on weekdays if more than 50 searches will be submitted.


If your queries are short (less than a few hundred bases) we suggest you merge them into one search of up to 1,000 bases


# Добавить
 ## поиск taxid по названию (выбирать по умолчанию один самый совпадающий?)
 https://ena-docs.readthedocs.io/en/latest/faq/taxonomy.html тут можно посмотреть удачный способ это сделать
 
 ## вещи которые мог менять древний апи бласта 
 https://ncbi.github.io/blast-cloud/dev/api.html

In [151]:
import urllib
import urllib3
import requests
import time
import re

from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

import pandas as pd

Пример использования:

    - последовательность: 
        "MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT",
    - база данных: Whole_Genome_Shotgun_contigs,
    - организм: Prevotella sp. oral taxon 472 str. F0295 (taxid:619693),
    - tblastn.

In [2]:
# поиск организма в базе данных WGS https://www.ncbi.nlm.nih.gov/Traces/wgs/

In [3]:
WGS_PROJECTS_URL = 'https://www.ncbi.nlm.nih.gov/Traces/wgs/index.cgi?'

In [4]:
WGS_payload = {
}

text:*Prevotellaceae* AND project_s:wgs AND (project_s:WGS) AND (project_s:WGS)

In [5]:
# генерация уникального имени - https://docs.python.org/3/library/uuid.html
import uuid

# make a random UUID
uuid.uuid4()

# make a UUID from a string of hex digits (braces and hyphens ignored)
x = uuid.UUID('{00010203-0405-0607-0809-0a0b0c0d0e0f}')

# рабочий простой вариант

# запрос в ncbi

In [6]:
fasta = \
""">line1
MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
"""

In [391]:
fasta = \
""">sf
MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIK
DAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
>line2
MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTE
>line3
RNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
>line4
FTWGRNPKVAMIETTEPYMDLRNPKVAMLSFDFFTWGRNPKVAMIETTE
>line5
ETTEVSKLSVESFKSTCYGYIICSYTYMDLRNPKVAMLSFDFFTWGRNPKVAMIETTIICSYTYMDL
RNPEIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
"""

In [392]:
len(fasta.strip('>').split('>'))

5

In [411]:
database = 'Whole_Genome_Shotgun_contigs'
taxon = '619693'


In [412]:
BLAST_URL = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"

In [475]:
payload = {
    'QUERY': fasta,
    'db': 'protein',
    'QUERYFILE': '',  # use for binary files
    'GENETIC_CODE': 1,
    'JOB_TITLE': 'fasta',
    'ADV_VIEW': 'true',
    'stype': 'nucleotide',
    #'SUBJECTFILE': '', # use for binary files
    'DATABASE': 'Whole_Genome_Shotgun_contigs',
    'DB_GROUP': 'wgsOrg',
    #'EQ_MENU': 'Prevotella sp. oral taxon 472 str. F0295 (taxid:619693)',  # работает если передавать полное название из выпадающего списка
    'EQ_MENU': '619693',    # работает и просто при передаче самого айдишника
    #'EQ_MENU': 'Prevotella sp. oral taxon 472 str. F0295',    # а так не работает
    #'NUM_ORG': 1,
    'MAX_NUM_SEQ': 100,
    'EXPECT': 0.05,
    'WORD_SIZE': 6,
    'HSP_RANGE_MAX': 0,
    'MATRIX_NAME': 'BLOSUM62',
    'MATCH_SCORES': [1, -2],
    'GAPCOSTS': '11 1',
    'COMPOSITION_BASED_STATISTICS': 2,
    'FILTER': 'L',
    'REPEATS': 566037,
    #'TEMPLATE_LENGTH': 0,
    #'TEMPLATE_TYPE': 0,
    #'PSSM': '(binary)',
    #'SHOW_OVERVIEW': 'true',
    #'SHOW_LINKOUT': 'true',
    'GET_SEQUENCE': 'true',
    'FORMAT_OBJECT': 'Alignment',
    'FORMAT_TYPE': 'HTML',
    'ALIGNMENT_VIEW': 'Pairwise',
    #'MASK_CHAR': 2,
    #'MASK_COLOR': 1,
    #'DESCRIPTIONS': 100,
    #'ALIGNMENTS': 100,
    #'LINE_LENGTH': 60,
    #'NEW_VIEW': 'true',
    'NCBI_GI': 'false',
    'SHOW_CDS_FEATURE': 'false',
    #'NUM_OVERVIEW': 100,
    #'FORMAT_NUM_ORG': 1,
    'CONFIG_DESCR': '2,3,4,5,8,9,10,11,12,13,14',
    'CLIENT': 'web',
    'SERVICE': 'plain',
    'CMD': 'request',
    'PAGE': 'Translations',
    'PROGRAM': 'tblastn',
    'UNGAPPED_ALIGNMENT': 'no',
    'BLAST_PROGRAMS': 'tblastn',
    'DB_DISPLAY_NAME': 'wgs',
    'ORG_DBS': 'orgDbsOnly_wgs',
    'SHOW_ORGANISMS': 'on',
    'SELECTED_PROG_TYPE': 'tblastn',
    'SAVED_SEARCH': 'true',
    'NUM_DIFFS': 1,
    'NUM_OPTS_DIFFS': 0,
    'PAGE_TYPE': 'BlastSearch',
    
}

In [490]:
prev_time = time.time()

resp = requests.post(BLAST_URL, data=payload)

soup = BeautifulSoup(resp.content)
resp.status_code

200

In [491]:
# Extract RID
RID = soup.find('input', {'name': 'RID'}).get('value')
RID

'2MFBPU3N013'

## Запрашиваем результат по риду, пока не получим его

In [492]:
payload = {
    'CMD': 'Get',
    'RID': RID,
    'FORMAT_TYPE': 'HTML'   
}

In [493]:
# stop when it takes more than 15 minutes.

delay = 10
wait = 10

i = 0

while True:
    
    current_time = time.time()
    wait = prev_time + delay - current_time
    
    if wait > 0:
        time.sleep(wait)
        prev_time = current_time + wait
        
    if i * delay >= 900:
        print(f'Oops! Try to check your resalts later (or restart). RID: {RID}.')
    
    else:
        prev_time = current_time
    
    
    try:
        resp = requests.post(BLAST_URL, data=payload)
        soup = BeautifulSoup(resp.content)
    
    except (ValueError, 
            urllib3.exceptions.InvalidChunkLength, 
            urllib3.exceptions.ProtocolError, 
            requests.exceptions.ChunkedEncodingError):
        print('Please wait. Searching.')
        i += 1
        continue
    
    else:
        if not soup.find('table', {'id': 'statInfo'}):
            print(f'status code: {resp.status_code}, DONE.')
            break
    

status code: 200, DONE.


In [494]:
database = re.match('.*\((.*)\)', soup.find(id="dbName").text).group(1)
database

'WGS_VDB://ACZS01'

In [495]:
seq_list = []

for line in soup.find_all('form', attrs={"id": "formBlastDescr"})[0].find_all('input', attrs={'type': 'checkbox'}):
    seq_list.append('gb|' + re.sub('Select seq ', '', line.next_sibling.text).split('.')[0] +'|')
    
seq_list

['gb|ACZS01000003|',
 'gb|ACZS01000174|',
 'gb|ACZS01000173|',
 'gb|ACZS01000087|']

In [496]:
def get_algnmt(RID, seq_list):
    
    FASTA_URL = 'https://www.ncbi.nlm.nih.gov/blast/SEQDUMP_SRA/seqdump.cgi'
    align_seq_list = ','.join(seq_list)
    
    params = {
        'CMD': 'Get',
        'RID': RID,
        #'DESCRIPTIONS': 0,
        #'NUM_OVERVIEW': 0,
        'GET_SEQUENCE': 'on',
        'DYNAMIC_FORMAT': 'on',
        'ALIGN_SEQ_LIST': align_seq_list,
        #'HSP_SORT': 0,
        'SEQ_LIST_START': 1,
        'QUERY_INDEX': 0,
        'ADV_VIEW': 'on',
        'SHOW_LINKOUT': 'on',
        #'MASK_CHAR': 2,
        #'MASK_COLOR': 1,
        'ALIGNMENT_VIEW': 'Pairwise',
        'LINE_LENGTH': 60,
        'BOBJSRVC': 'sra'
        }
    
    resp = requests.get('https://blast.ncbi.nlm.nih.gov/t2g.cgi',params=params)
    
    return resp

In [497]:
filename = "page2.html"

resp = get_algnmt(RID, seq_list)
resp.status_code

200

In [498]:
soup_2 = BeautifulSoup(resp.content)

## Приводим результаты в читаемый вид

In [499]:
class Alignment():
    
    def __init__(self,
                 subj_name, subj_id, subj_len, subj_range,
                 score, e_value, identity, 
                 query_seq, subj_seq):
        '''
        Создает объект класса Alignment со следующими атрибутами:
        self.score
        self.subj_range    # tuple
        self.subj_name
        self.subj_id
        self.subj_len
        self.e_value
        self.identity
        self.query_seq
        self.subj_seq
        '''
        
        self.score = score
        self.subj_range  = subj_range   # tuple
        self.subj_name = subj_name
        self.subj_id = subj_id
        self.subj_len = subj_len
        self.e_value = e_value
        self.identity = identity
        self.query_seq = query_seq
        self.subj_seq = subj_seq
        
        
    def __str__(self):
        return ': '.join([self.subj_id, self.score])


In [500]:
four = soup_2.find_all('div', {'class': 'oneSeqAln'})

alignments_list = []

for element in four:
    
    id_name = element.find('div', {'class': 'dlfRow'})
    
    subj_name = id_name.text.strip().split('\n')[0]    
    subj_len = int(element.find("label", text="Length: ").next_sibling.text)
    subj_id = id_name.find('a').text
    
    alignments = element.find_all('div', {'class': 'alnAll'})
    
    for align in alignments:
        
        table_params = align.find_all('table', {'class' : "alnParams"})
        nember_al = len(table_params)
        urls = align.find_all('a', {'class' : ""}, href=True)
        
        sequences = align.find_all('pre')
        
        for i in range(nember_al):
            parsed_url = urlparse(urls[i]['href'])
            
            scores_list = pd.read_html(str(table_params[i]))
            df_scores = scores_list[0]
            
            score = df_scores['Score'][0]
            identity = df_scores['Identities'][0]
            e_value =  df_scores['Expect'][0]
            
            # more
            method = df_scores['Method'][0]
            positives = df_scores['Positives'][0]
            gaps = df_scores['Gaps'][0]
            frame = df_scores['Frame'][0]
            
            value_from = int(parse_qs(parsed_url.query)['from'][0])
            value_to = int(parse_qs(parsed_url.query)['to'][0])
            subj_range = tuple([value_from, value_to])
            
            query_seq = []
            subj_seq = []
            
            al = sequences[i].text.strip().split('\n')
            
            for j, line in enumerate(al):
                line = line.split()
                if 'Query' in line:
                    query_seq.append(line[2])
                    
                elif 'Sbjct' in line:
                    subj_seq.append(line[2])
            
            query_seq = ''.join(query_seq)
            subj_seq = ''.join(subj_seq)
            
            
            alignments_list.append(Alignment(subj_name=subj_name,
                                             subj_id=subj_id,
                                             subj_len=subj_len,
                                             subj_range=subj_range,
                                             score=score,
                                             e_value=e_value,
                                             identity=identity,
                                             query_seq=query_seq,
                                             subj_seq=subj_seq))
        
for el in alignments_list:
    print(el)
    print(el.identity)

ACZS01000003.1: 187 bits(474)
89/89(100%)
ACZS01000003.1: 34.7 bits(78)
18/41(44%)
ACZS01000174.1: 77.8 bits(190)
39/86(45%)
ACZS01000173.1: 75.5 bits(184)
40/87(46%)
ACZS01000173.1: 73.6 bits(179)
37/82(45%)
ACZS01000087.1: 47.4 bits(111)
21/34(62%)


In [378]:
def save_content(resp, filename):
    with open(filename, "wb") as file:
        file.write(resp.content)
        
save_content(resp, "page.html")

In [305]:
# get subj_name

subj_name_list = []

first = soup.find_all('form', attrs={"id": "formBlastDescr"})[0].find_all('div', attrs={'id': 'scrollBody'})


for element in first[0].find_all('a', attrs={"class": "deflnDesc"}):
    subj_name_list.append(element.text)
    
subj_name_list

['Prevotella sp. oral taxon 472 str. F0295 contig00003, whole genome shotgun sequence',
 'Prevotella sp. oral taxon 472 str. F0295 contig00239, whole genome shotgun sequence',
 'Prevotella sp. oral taxon 472 str. F0295 contig00238, whole genome shotgun sequence',
 'Prevotella sp. oral taxon 472 str. F0295 contig00095, whole genome shotgun sequence']

In [306]:
# get subj_id:

subj_id_list = []

for line in first[0].find_all('td', attrs={"class": "c12"}):
    subj_id_list.append(line.text.strip())
    
subj_id_list

['ACZS01000003.1', 'ACZS01000174.1', 'ACZS01000173.1', 'ACZS01000087.1']

In [307]:
# e_value

e_value_list = []

for element in first[0].find_all('td', attrs={"class": "c9"}):
    e_value_list.append(element.text)
    
e_value_list

['1e-57', '2e-19', '2e-18', '1e-08']

In [308]:
# e_value - в текстовом формате!

identity_list = []

for element in first[0].find_all('td', attrs={"class": "c10"}):
    identity_list.append(element.text)
    
identity_list

['100.00%', '45.35%', '45.98%', '61.76%']

In [309]:
# subj_len

subj_len_list = []

for element in first[0].find_all('td', attrs={"class": "c11"}):
    subj_len_list.append(element.text)
    
subj_len_list

['10342', '2654', '1451', '1187']

In [310]:
# subj_len

score_list = []

for element in first[0].find_all('td', attrs={"class": "c6"}):
    score_list.append(element.text)
    
score_list

['', '187', '', '77.8', '', '75.5', '', '47.4']

#### Другой путь

In [311]:
# get subj_range

subj_range_list = []

second = soup_2.find_all('div', {'class': 'alnAll'})

for element in second:
    for i in element.find_all('span', {'class': 'alnRn'}):
        numbers = re.match(r'[0-9]+ to [0-9]+', i.text.split(': ')[1]).group(0).split(' to ')
        subj_range_list.append(tuple(numbers))
        
subj_range_list

[('4187', '4453'),
 ('3495', '3617'),
 ('286', '543'),
 ('98', '346'),
 ('1021', '1266'),
 ('903', '1004')]

In [312]:
len(second)

4

In [313]:
# get subj_name, subj_id, subj_len

subj_name_list = []
subj_id_list = []
subj_len_list = []

trird = soup_2.find_all('div', {'class': 'dlfRow'})

for element in trird:
    string_split = element.text.strip().split('\n')
    
    subj_name_list.append(string_split[0])
    
    for label in element.find_all("label", text="Sequence ID: "):
        subj_id_list.append(label.next_sibling.text)
    
    for label in element.find_all("label", text="Length: "):
        subj_len_list.append(label.next_sibling.text)
        
